## Merging .csv Files

Import the Pandas directory

In [6]:
import pandas as pd

Load 2 csv files (The Template and Questions) 

In [ ]:
# Load the GPTFuzzer.csv file
sentences = pd.read_csv('GPTFuzzer.csv')

# Load the question_list.csv file
replacements = pd.read_csv('question_list.csv')

# Create a new dataframe to store the output
output = pd.DataFrame()

The GPTFuzzer.csv template has \[INSERT PROMPT HERE] inside. Replace the sentence with the contents in "question_list.csv" and save it to all_questions.csv

In [ ]:
# Loop through each sentence
for index, sentence in sentences.iterrows():

    # Create a new row for the output dataframe
    new_row = sentence.copy()

    # Loop through each replacement
    for replacement in replacements.iloc[:, 1:].iloc[:, 0]:  

        # Replace the [INSERT PROMPT HERE] with the replacement
        new_row['text'] = sentence['text'].replace('[INSERT PROMPT HERE]', replacement)
        
        # Add the new row to the output dataframe
        output = pd.concat([output, pd.DataFrame([new_row])], ignore_index=True)

# Create a new 'id' column with sequential values
output['id'] = range(len(output))

# Save the output to a new CSV file
output.to_csv('all_questions.csv', index=False)


Split the new csv files into parts of 10.

This is useful when a timeout has occured, and you only need to modify the code a bit and continue from the last timeout

In [ ]:
# Load the all_questions.csv file
df = pd.read_csv('all_questions.csv')

# Define how many lists in each split
split_size = 10

# Create a list to store the splitted DataFrames
split_dfs = []

# Split the DataFrame
for i in range(0, len(df), split_size):
    split = df.iloc[i:i+split_size]
    split_dfs.append(split)

# Save each split to a new csv file (This will be named q{1, 2,...}.csv)
for i, split in enumerate(split_dfs):
    split.to_csv(f'q{i+1}.csv', index=False)

## Setting Up Gemini AI

Install the Google AI SDK in cmd:

pip install google-generativeai

Import necessary libraries

In [ ]:
import time
import google.generativeai as genai

Get and use the API key to be able to use Gemini

Getting the API key:

https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fmakersuite.google.com%2Fapp%2Fapikey

In [ ]:
GOOGLE_API_KEY="[INSERT THE API KEY HERE]"

genai.configure(api_key=GOOGLE_API_KEY)

In this one, I am choosing gemini-1.5-flash as the Model, but when using the Paid version, I recommend to use gemini-1.5-pro instead.

For a brief reasoning:

For free users, gemini-1.5-flash gives 15 Requests per Minute, and a maximum of 1500 Requests per Day.

And the free users using gemini-1.5-pro gets only 2 Requests per Minute.

Paid users have very high limits, which makes it more efficient doing these stuffs.

source: https://ai.google.dev/pricing

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')

When using the **Free** version of Gemini, the prompt input is limited to 15 requests per minutes, and it has a limit of 1500 requests per day. To bypass this, I use a 60-second timer for each 1 csv file that has been answered.

In [ ]:
#Set the index number
num = 1

for i in range(769):

    #Load all the q.csv files
    df = pd.read_csv(f'q{1+i}.csv')

    #Loop the lists inside the csv file
    for index, row in df.iterrows():
        prompt = row.iloc[1]

        #Generate the response
        response = model.generate_content(prompt)

        if response.parts:
            answer = response.parts[0].text
        else:
            answer = "\"No response generated\""
        
        #Insert the output to a new "answer_gemini.csv" file
        answer_df = pd.DataFrame({'id': [num], 'answer': [answer]})
        answer_df.to_csv(f'answer_gemini.csv', mode='a', header=False, index=False)

        #Add the next index number
        num += 1
    
    # Wait for 60 seconds
    time.sleep(60)

When using the **Paid** version, it doesnt need any timer to delay, so it is recommended to run this code instead.

In [ ]:
#Set the index number
num = 1

for i in range(769):

    #Load all the q.csv files
    df = pd.read_csv(f'q{1+i}.csv')

    #Loop the lists inside the csv file
    for index, row in df.iterrows():
        prompt = row.iloc[1]

        #Generate the response
        response = model.generate_content(prompt)

        if response.parts:
            answer = response.parts[0].text
        else:
            answer = "\"No response generated\""
        
        #Insert the output to a new "answer_gemini.csv" file
        answer_df = pd.DataFrame({'id': [num], 'answer': [answer]})
        answer_df.to_csv(f'answer_gemini.csv', mode='a', header=False, index=False)

        #Add the next index number
        num += 1